In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ="/home/tdelatte/new-projects/ethereum-analytics/key/ethereum-analytics-309308-6c01508bc0b8.json"

In [2]:
from google.cloud import bigquery
client = bigquery.Client()

In [3]:
import pandas as pd

In [4]:
def load_data_from_bigquery(QUERY):
    
    query_job = client.query(QUERY) # API request
    df = query_job.to_dataframe()
    
    return df

In [5]:
QUERY = """

        WITH
          ethereum_balance AS (
          SELECT
            address AS ethereum_address,
            (eth_balance / POWER(10, 18)) AS eth_balance
          FROM
            `bigquery-public-data.crypto_ethereum.balances`
          WHERE
            (eth_balance / POWER(10, 18)) > 100
          LIMIT
            100000),
          
          top_tokens AS (
          SELECT
            token_address,
            COUNT(1) AS transfer_count
          FROM
            `bigquery-public-data.ethereum_blockchain.token_transfers` AS token_transfers
          GROUP BY
            token_address
          ORDER BY
            transfer_count DESC
          LIMIT
            100000),
         
         token_balances AS (
          WITH
            double_entry_book AS (
            SELECT
              token_address,
              to_address AS ethereum_address,
              CAST(value AS float64) AS value,
              block_timestamp,
              transaction_hash
            FROM
              `bigquery-public-data.ethereum_blockchain.token_transfers`
            UNION ALL
            SELECT
              token_address,
              from_address AS ethereum_address,
              -CAST(value AS float64) AS value,
              block_timestamp,
              transaction_hash
            FROM
              `bigquery-public-data.ethereum_blockchain.token_transfers` )
          SELECT
            a.ethereum_address,
            b.token_address,
            SUM(value) AS balance,
            COUNT(DISTINCT transaction_hash) as unique_transfers
          FROM
            ethereum_balance a
          JOIN
            double_entry_book b
          ON
            a.ethereum_address = b.ethereum_address
          JOIN
            top_tokens c
          ON
            c.token_address = b.token_address
          WHERE
            a.ethereum_address != '0x0000000000000000000000000000000000000000'
          GROUP BY
            1,
            2
          HAVING
            balance > 0 )
            
            
        SELECT
          ethereum_address,
          MAX(eth_balance) AS ether_balance,
          COUNT(DISTINCT token_address) AS unique_tokens,
          MAX(unique_transfers) AS unique_transfers
        FROM
          ethereum_balance a
        JOIN
          token_balances b
        USING
          (ethereum_address)
        GROUP BY
          1

    """

In [6]:
eth_dataset = load_data_from_bigquery(QUERY)

In [7]:
eth_dataset.head(10)

,ethereum_address,ether_balance,unique_tokens,unique_transfers
0,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,17960.597685,9900,117342
1,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,19345.149872,1718,54180
2,0x0d0707963952f2fba59dd06f2b425ace40b492fe,443.735122,587,245377
3,0xab5c66752a9e8167967685f1450532fb96d5d24f,331.840176,402,443233
4,0x1ce7ae555139c5ef5a57cc8d814a867ee6ee33d8,747.701162,978,18360
5,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,1031.186386,865,472190
6,0x564286362092d8e7936f0549571a803b203aaced,23892.712593,502,109404
7,0x0016eccecffc25b94050187017eb59fa05c029aa,126.407467,54,6180
8,0xbe708d227f6dfa0b8f2698bf543b949dfe4e28fb,269.029806,202,1462
9,0x9b77ab003d44b9b9cb47fa6a00276a23c05b49a5,2089.859796,54,3


In [8]:
from etherscan import Etherscan
api_key = os.environ.get("ETHERSCAN_API_KEY")
eth = Etherscan(api_key) # key in quotation marks

In [9]:
eth = Etherscan("UJZSE9DXM1T37JZQVNQYNSS9IT5JEPIBA7") # key in quotation marks

In [10]:
eth_dataset["mined_blocks"] = 0

In [11]:
eth_dataset.head()

,ethereum_address,ether_balance,unique_tokens,unique_transfers,mined_blocks
0,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,17960.597685,9900,117342,0
1,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,19345.149872,1718,54180,0
2,0x0d0707963952f2fba59dd06f2b425ace40b492fe,443.735122,587,245377,0
3,0xab5c66752a9e8167967685f1450532fb96d5d24f,331.840176,402,443233,0
4,0x1ce7ae555139c5ef5a57cc8d814a867ee6ee33d8,747.701162,978,18360,0


In [20]:
def add_mined_blocks(df):
    for i, row in df.iterrows():
        eth_address = row.ethereum_address
        try:
            mined = len(eth.get_mined_blocks_by_address(address=eth_address))
        except:
            continue
        if mined:
            df.iat[i-1, 4] = mined
    return df

In [21]:
add_mined_blocks(eth_dataset)

,ethereum_address,ether_balance,unique_tokens,unique_transfers,mined_blocks
0,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,17960.597685,9900,117342,0
1,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,19345.149872,1718,54180,0
2,0x0d0707963952f2fba59dd06f2b425ace40b492fe,443.735122,587,245377,0
3,0xab5c66752a9e8167967685f1450532fb96d5d24f,331.840176,402,443233,0
4,0x1ce7ae555139c5ef5a57cc8d814a867ee6ee33d8,747.701162,978,18360,0
...,...,...,...,...,...
26524,0x4d5a77b869312a8e2b3daa67c01c59753f6254ae,281.630258,47,12,0
26525,0xd2ad5f590192d227cb7e6ff15743119fe26118a0,1000.954777,47,4,0
26526,0x603f39c81560019c8360f33ba45bc1e4caecb33e,176.012588,47,7,0
26527,0x003e93083a2d294cb8c4421048108330c37b5874,669.846139,47,2,0


In [22]:
eth_dataset["mined_blocks"].value_counts()

0         26412
1             6
2             6
5             4
3             3
          ...  
89            1
48686         1
282304        1
10            1
23            1
Name: mined_blocks, Length: 85, dtype: int64

In [26]:
eth_dataset_complete = Out[21]

In [28]:
eth_dataset_complete.to_csv("../data/processed/eth_dataset_complete.csv")

In [27]:
eth_dataset_complete.shape

(26529, 5)

In [14]:
# Sanity check: do eth_addresses labeled as "Miners" actually have mined_blocks

In [29]:
labeled_dataset = pd.read_csv("../data/processed/eth_addresses_labels.csv")
labeled_dataset = labeled_dataset[labeled_dataset.Entity != "Entity"]

In [30]:
labeled_dataset.rename({'Address': 'ethereum_address'}, axis=1, inplace=True)

In [35]:
eth_dataset_complete["Entity"] = "Unknown"

In [39]:
eth_dataset_complete.drop(["Label"], axis=1)

,ethereum_address,ether_balance,unique_tokens,unique_transfers,mined_blocks,Entity
0,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,17960.597685,9900,117342,0,Unknown
1,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,19345.149872,1718,54180,0,Unknown
2,0x0d0707963952f2fba59dd06f2b425ace40b492fe,443.735122,587,245377,0,Unknown
3,0xab5c66752a9e8167967685f1450532fb96d5d24f,331.840176,402,443233,0,Unknown
4,0x1ce7ae555139c5ef5a57cc8d814a867ee6ee33d8,747.701162,978,18360,0,Unknown
...,...,...,...,...,...,...
26524,0x4d5a77b869312a8e2b3daa67c01c59753f6254ae,281.630258,47,12,0,Unknown
26525,0xd2ad5f590192d227cb7e6ff15743119fe26118a0,1000.954777,47,4,0,Unknown
26526,0x603f39c81560019c8360f33ba45bc1e4caecb33e,176.012588,47,7,0,Unknown
26527,0x003e93083a2d294cb8c4421048108330c37b5874,669.846139,47,2,0,Unknown


In [45]:
eth_dataset_complete = eth_dataset_complete.drop(["Label"], axis=1)

In [48]:
eth_dataset_complete.shape

(26529, 6)

In [47]:
labeled_dataset.head()

,ethereum_address,Entity
0,0x8ab7404063ec4dbcfd4598215992dc3f8ec853d7,DeFi
1,0x1c74cff0376fb4031cd7492cd6db2d66c3f2c6b9,DeFi
2,0x06af07097c9eeb7fd685c692751d5c66db49c215,DeFi
3,0xc00e94cb662c3520282e6f5717214004a7f26888,DeFi
4,0xb3319f5d18bc0d84dd1b4825dcde5d5f7266d407,DeFi


In [51]:
labeled_dataset_complete = pd.merge(eth_dataset_complete, labeled_dataset, how="left", on="ethereum_address")

In [57]:
labeled_dataset.shape

(19137, 2)

In [56]:
labeled_dataset_complete.head(50)

,ethereum_address,ether_balance,unique_tokens,unique_transfers,mined_blocks,Entity_x,Entity_y
0,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,17960.597685,9900,117342,0,Unknown,Dex
1,0x2a0c0dbecc7e4d658f48e01e3fa353f44050c208,19345.149872,1718,54180,0,Unknown,Dex
2,0x0d0707963952f2fba59dd06f2b425ace40b492fe,443.735122,587,245377,0,Unknown,Exchange
3,0xab5c66752a9e8167967685f1450532fb96d5d24f,331.840176,402,443233,0,Unknown,Exchange
4,0x1ce7ae555139c5ef5a57cc8d814a867ee6ee33d8,747.701162,978,18360,0,Unknown,Dex
5,0x6cc5f688a315f3dc28a7781717a9a798a59fda7b,1031.186386,865,472190,0,Unknown,Exchange
6,0x564286362092d8e7936f0549571a803b203aaced,23892.712593,502,109404,0,Unknown,Exchange
7,0x0016eccecffc25b94050187017eb59fa05c029aa,126.407467,54,6180,0,Unknown,NaN
8,0xbe708d227f6dfa0b8f2698bf543b949dfe4e28fb,269.029806,202,1462,0,Unknown,NaN
9,0x9b77ab003d44b9b9cb47fa6a00276a23c05b49a5,2089.859796,54,3,0,Unknown,NaN


In [58]:
labeled_dataset_complete.Entity_y.value_counts()

Exchange       86
ICO Wallets    33
Mining         16
DeFi           10
Dex             9
Name: Entity_y, dtype: int64

In [59]:
labeled_dataset_complete.to_csv("../data/processed/labeled_dataset_complete.csv")

In [ ]:
# Manually add crowdsourced labels from etherscan.
# Cannot scrape data from Etherscan: CloudFare protection.

